In [1]:
### polyphase channelizer
import numpy as np
from scipy.signal import firwin, lfilter, freqz
from scipy.signal import butter, filtfilt, resample
from scipy.fft import fft, ifft, fftshift

import plotly.graph_objects as go
import plotly.express as px

In [2]:
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)  # zero-phase filtering
    return y

In [3]:
x_f_1 = np.zeros(1000) + 0j
x_f_1[0:400] = 1 + 0j
x_f_1[-400:] = 1 + 0j
x_f_2 = np.zeros(1000) + 0j
x_f_2[0:400] = 1 + 0j
x_f_2[-400:] = 1 + 0j
x_f_3 = np.zeros(1000) + 0j
x_f_3[0:400] = 1 + 0j
x_f_3[-400:] = 1 + 0j
x_f_4 = np.zeros(1000) + 0j
x_f_4[0:400] = 1 + 0j
x_f_4[-400:] = 1 + 0j

x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_t_4 = ifft(x_f_4)*len(x_f_4)

In [4]:
x_f_1 = np.random.randn(1000) + 0j
x_f_1[100:900] *= 1e-6
x_f_1_original = x_f_1.copy()
x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_1 = resample(x_t_1, 4000)

x_f_2 = np.random.randn(1000) + 0j
x_f_2[200:800] *= 1e-6
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_f_2_original = x_f_2.copy()
x_t_2 = resample(x_t_2, 4000)
x_t_2 *= np.exp(1j*2*np.pi*1000*np.arange(len(x_t_2))/4000)

x_f_3 = np.random.randn(1000) + 0j
x_f_3[300:700] *= 1e-6
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_f_3_original = x_f_3.copy()
x_t_3 = resample(x_t_3, 4000)
x_t_3 *= np.exp(1j*2*np.pi*2000*np.arange(len(x_t_3))/4000)

x_f_4 = np.random.randn(1000) + 0j
x_f_4[400:600] *= 1e-6
x_t_4 = ifft(x_f_4)*len(x_f_4)
x_f_4_original = x_f_4.copy()
x_t_4 = resample(x_t_4, 4000)
x_t_4 *= np.exp(1j*2*np.pi*3000*np.arange(len(x_t_4))/4000)

x_f_1 = fft(x_t_1)/len(x_t_1)
x_f_2 = fft(x_t_2)/len(x_t_1)
x_f_3 = fft(x_t_3)/len(x_t_1)
x_f_4 = fft(x_t_4)/len(x_t_1)

x_f_total = x_f_1 + x_f_2 + x_f_3 + x_f_4

x_t_total = ifft(x_f_total)*len(x_f_total)


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total)), mode='lines', name=f'Combined Wide-band Signal', line=dict(dash="dot")))    
fig.update_layout(width=700,height=400, title='Multi-Channel  Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [6]:
fig.write_html("wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

# Conventional Approach

In [7]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

num_taps = 64  # Number of filter taps (coefficients) - typically odd for linear phase

filter_taps = signal.firwin(num_taps, 0.25, pass_zero=True)

filter_freq_response = fft(filter_taps, 4000)

In [8]:
fig_filter = go.Figure()
fig_filter.add_trace(go.Scatter(y=filter_taps, mode='lines+markers', name=f'64-Tap Low-pass Filter'))   
fig_filter.update_layout(width=700,height=400, title='64-Tap Low-pass Filter', xaxis_title='Time Sample', yaxis_title='Magnitude (Linear)',
                  )

In [9]:
fig_filter.write_html("lpf_taps.html",
                        include_plotlyjs='cdn',
                        full_html=False)

In [10]:
### Apply frequency shift on channel 3
x_f_total_shifted = np.roll(x_f_total, -2000)
x_f_1_shifted = np.roll(x_f_1, -2000)
x_f_2_shifted = np.roll(x_f_2, -2000)
x_f_3_shifted = np.roll(x_f_3, -2000)
x_f_4_shifted = np.roll(x_f_4, -2000)

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1_shifted)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2_shifted)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_shifted)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4_shifted)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_shifted)), mode='lines', name=f'Wide-band Signal (Frequency-Shifted)', line=dict(dash="dot")))   
fig.update_layout(width=700,height=400, title='Frequency-Shifted Multi-Channel Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [12]:
fig.write_html("shifted_wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

In [13]:
# 4. Apply the filter to the signal
x_f_total_filtered = filter_freq_response*x_f_total_shifted
x_f_1_filtered = filter_freq_response*x_f_1_shifted
x_f_2_filtered = filter_freq_response*x_f_2_shifted
x_f_3_filtered = filter_freq_response*x_f_3_shifted
x_f_4_filtered = filter_freq_response*x_f_4_shifted

In [14]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1_filtered)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2_filtered)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_filtered)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4_filtered)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_filtered)), mode='lines', name=f'Wide-band Signal (Filtered)', line=dict(dash="dot")))   
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(filter_freq_response)), mode='lines', name=f'Filter Response', line=dict(dash="dot")))   
fig.update_layout(width=700,height=400, title='Filtered Multi-Channel Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [15]:
fig.write_html("filtered_wideband_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

In [16]:
x_t_total_filtered = ifft(x_f_total_filtered)*len(x_f_total_filtered)
x_t_total_filtered_downsamp = x_t_total_filtered[::4]
x_f_total_filtered_downsamp = fft(x_t_total_filtered_downsamp)/len(x_t_total_filtered_downsamp)

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total_filtered_downsamp)), mode='lines', name=f'Signal 3 (Extracted)'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3_original)), mode='lines', name=f'Signal 3 (Original)'))    
fig.update_layout(width=700,height=400, title='Signal 3 Extracted by Decimation Filtering', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 1000], yaxis_range=[-150, 10])

In [18]:
fig.write_html("decim_filtered_signal.html",
               include_plotlyjs='cdn',
               full_html=False)

# Polyphase Channelizer

In [172]:
x_f_1 = np.ones(1000) + 0j
x_f_1[100:900] *= 1e-6
x_f_1_original = x_f_1.copy()
x_t_1 = ifft(x_f_1)*len(x_f_1)
x_t_1 = resample(x_t_1, 4000)

x_f_2 = np.ones(1000) + 0j
x_f_2[200:800] *= 1e-6
x_t_2 = ifft(x_f_2)*len(x_f_2)
x_f_2_original = x_f_2.copy()
x_t_2 = resample(x_t_2, 4000)
x_t_2 *= np.exp(1j*2*np.pi*1000*np.arange(len(x_t_2))/4000)

x_f_3 = np.ones(1000) + 0j
x_f_3[300:700] *= 1e-6
x_t_3 = ifft(x_f_3)*len(x_f_3)
x_f_3_original = x_f_3.copy()
x_t_3 = resample(x_t_3, 4000)
x_t_3 *= np.exp(1j*2*np.pi*2000*np.arange(len(x_t_3))/4000)

x_f_4 = np.ones(1000) + 0j
x_f_4[400:600] *= 1e-6
x_t_4 = ifft(x_f_4)*len(x_f_4)
x_f_4_original = x_f_4.copy()
x_t_4 = resample(x_t_4, 4000)
x_t_4 *= np.exp(1j*2*np.pi*3000*np.arange(len(x_t_4))/4000)

x_f_1 = fft(x_t_1)/len(x_t_1)
x_f_2 = fft(x_t_2)/len(x_t_1)
x_f_3 = fft(x_t_3)/len(x_t_1)
x_f_4 = fft(x_t_4)/len(x_t_1)

x_f_total = x_f_1 + x_f_2 + x_f_3 + x_f_4

x_t_total = ifft(x_f_total)*len(x_f_total)


In [173]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_1)), mode='lines', name=f'Signal 1 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_2)), mode='lines', name=f'Signal 2 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_3)), mode='lines', name=f'Signal 3 Contribution'))    
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_4)), mode='lines', name=f'Signal 4 Contribution'))
fig.add_trace(go.Scatter(y=20*np.log10(np.abs(x_f_total)), mode='lines', name=f'Combined Wide-band Signal', line=dict(dash="dot")))    
fig.update_layout(width=700,height=400, title='Multi-Channel  Waveform', xaxis_title='Frequency Sample', yaxis_title='Magnitude (dB)',
                  xaxis_range=[0, 4000], yaxis_range=[-150, 10])

In [21]:
fig.write_html("wideband_signal_polyphase_input.html",
               include_plotlyjs='cdn',
               full_html=False)

In [22]:
### downsample into 4 staggered streams
# stream 1
x_f_total[abs(x_f_total)<1e-3] = np.nan
x_f_1[abs(x_f_1)<1e-3] = np.nan
x_f_2[abs(x_f_2)<1e-3] = np.nan
x_f_3[abs(x_f_3)<1e-3] = np.nan
x_f_4[abs(x_f_4)<1e-3] = np.nan

idx = np.arange(4000)


fig1 = go.Figure()
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1),
                           z=np.imag(x_f_1),
                           mode='markers', name=f'Channel 1 Contribution',
                           marker=dict(size=5,symbol="circle"),
                           opacity=0.5))    
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2),
                           z=np.imag(x_f_2),
                           mode='markers', name=f'Channel 2 Contribution',
                           marker=dict(size=5,symbol="square" ),
                           opacity=0.5))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3),
                           z=np.imag(x_f_3),
                           mode='markers', name=f'Channel 3 Contribution',
                           marker=dict(size=5,symbol="diamond" ),
                           opacity=0.5))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4),
                           z=np.imag(x_f_4),
                           mode='markers', name=f'Channel 4 Contribution',
                           marker=dict(size=5,symbol="cross"),
                           opacity=0.5))


fig1.update_layout(width=700,height=650,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1, 1], title="I"),
        zaxis=dict(range=[-1, 1], title="Q"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.75)),  # adjust the camera so labels are visible

    ),
    title="Complex Spectrum of Wide-band Input Signal",
    legend=dict(
        orientation="h",          # horizontal
        yanchor="bottom",         # anchor legend to bottom
        y=0,                   # push legend below plot
        xanchor="center",         # anchor centered horizontally
        x=0.5
    )
)

# Zero axes

fig1.add_trace(go.Scatter3d(
    x=[0,4000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[4000,4000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[4000,4000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig1.show()

In [23]:
fig1.write_html("wideband_signal_polyphase_input_complex.html",
               include_plotlyjs='cdn',
               full_html=False)

In [101]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_1 = x_t_total[0::4]
x_t_1_downsamp_1 = x_t_1[0::4]
x_t_2_downsamp_1 = x_t_2[0::4]
x_t_3_downsamp_1 = x_t_3[0::4]
x_t_4_downsamp_1 = x_t_4[0::4]

x_f_total_downsamp_1 = fft(x_t_total_downsamp_1)/len(x_t_total_downsamp_1)
x_f_1_downsamp_1 = fft(x_t_1_downsamp_1)/len(x_t_1_downsamp_1)
x_f_2_downsamp_1 = fft(x_t_2_downsamp_1)/len(x_t_2_downsamp_1)
x_f_3_downsamp_1 = fft(x_t_3_downsamp_1)/len(x_t_3_downsamp_1)
x_f_4_downsamp_1 = fft(x_t_4_downsamp_1)/len(x_t_4_downsamp_1)

x_f_total_downsamp_1[abs(x_f_total_downsamp_1)<1e-3] = np.nan
x_f_1_downsamp_1[abs(x_f_1_downsamp_1)<1e-3] = np.nan
x_f_2_downsamp_1[abs(x_f_2_downsamp_1)<1e-3] = np.nan
x_f_3_downsamp_1[abs(x_f_3_downsamp_1)<1e-3] = np.nan
x_f_4_downsamp_1[abs(x_f_4_downsamp_1)<1e-3] = np.nan


fig1 = go.Figure()
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_1),
                           z=np.imag(x_f_1_downsamp_1),
                           mode='markers', name=f'Channel 1',
                           marker=dict(size=5, symbol="circle")))    
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_1),
                           z=np.imag(x_f_2_downsamp_1),
                           mode='markers', name=f'Channel 2',
                           marker=dict(size=5, symbol="square")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_1),
                           z=np.imag(x_f_3_downsamp_1),
                           mode='markers', name=f'Channel 3',
                           marker=dict(size=5, symbol="diamond")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_1),
                           z=np.imag(x_f_4_downsamp_1),
                           mode='markers', name=f'Channel 4',
                           marker=dict(size=5, symbol="cross")))


fig1.update_layout(width=1000,height=600,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1, 1], title="Y"),
        zaxis=dict(range=[-1, 1], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.75)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 1 (delay=0)"
)

# Zero axes

fig1.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig1.show()

In [102]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_2 = x_t_total[1::4]
x_t_1_downsamp_2 = x_t_1[1::4]
x_t_2_downsamp_2 = x_t_2[1::4]
x_t_3_downsamp_2 = x_t_3[1::4]
x_t_4_downsamp_2 = x_t_4[1::4]

x_f_total_downsamp_2 = fft(x_t_total_downsamp_2)/len(x_t_total_downsamp_2)
x_f_1_downsamp_2 = fft(x_t_1_downsamp_2)/len(x_t_1_downsamp_2)
x_f_2_downsamp_2 = fft(x_t_2_downsamp_2)/len(x_t_2_downsamp_2)
x_f_3_downsamp_2 = fft(x_t_3_downsamp_2)/len(x_t_3_downsamp_2)
x_f_4_downsamp_2 = fft(x_t_4_downsamp_2)/len(x_t_4_downsamp_2)

x_f_total_downsamp_2[abs(x_f_total_downsamp_2)<1e-3] = np.nan
x_f_1_downsamp_2[abs(x_f_1_downsamp_2)<1e-3] = np.nan
x_f_2_downsamp_2[abs(x_f_2_downsamp_2)<1e-3] = np.nan
x_f_3_downsamp_2[abs(x_f_3_downsamp_2)<1e-3] = np.nan
x_f_4_downsamp_2[abs(x_f_4_downsamp_2)<1e-3] = np.nan


fig2 = go.Figure()
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_2),
                           z=np.imag(x_f_1_downsamp_2),
                           mode='markers', name=f'Channel 1',
                           marker=dict(size=5, symbol="circle")))    
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_2),
                           z=np.imag(x_f_2_downsamp_2),
                           mode='markers', name=f'Channel 2',
                           marker=dict(size=5, symbol="square")))  
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_2),
                           z=np.imag(x_f_3_downsamp_2),
                           mode='markers', name=f'Channel 3',
                           marker=dict(size=5, symbol="diamond")))  
fig2.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_2),
                           z=np.imag(x_f_4_downsamp_2),
                           mode='markers', name=f'Channel 4',
                           marker=dict(size=5, symbol="cross")))  


fig2.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 2 (delay=1)"
)

# Zero axes

fig2.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig2.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig2.show()

In [103]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_3 = x_t_total[2::4]
x_t_1_downsamp_3 = x_t_1[2::4]
x_t_2_downsamp_3 = x_t_2[2::4]
x_t_3_downsamp_3 = x_t_3[2::4]
x_t_4_downsamp_3 = x_t_4[2::4]

x_f_total_downsamp_3 = fft(x_t_total_downsamp_3)/len(x_t_total_downsamp_3)
x_f_1_downsamp_3 = fft(x_t_1_downsamp_3)/len(x_t_1_downsamp_3)
x_f_2_downsamp_3 = fft(x_t_2_downsamp_3)/len(x_t_2_downsamp_3)
x_f_3_downsamp_3 = fft(x_t_3_downsamp_3)/len(x_t_3_downsamp_3)
x_f_4_downsamp_3 = fft(x_t_4_downsamp_3)/len(x_t_4_downsamp_3)

x_f_total_downsamp_3[abs(x_f_total_downsamp_3)<1e-3] = np.nan
x_f_1_downsamp_3[abs(x_f_1_downsamp_3)<1e-3] = np.nan
x_f_2_downsamp_3[abs(x_f_2_downsamp_3)<1e-3] = np.nan
x_f_3_downsamp_3[abs(x_f_3_downsamp_3)<1e-3] = np.nan
x_f_4_downsamp_3[abs(x_f_4_downsamp_3)<1e-3] = np.nan


fig3 = go.Figure()
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_3),
                           z=np.imag(x_f_1_downsamp_3),
                           mode='markers', name=f'Channel 1',
                           marker=dict(size=5, symbol="circle")))    
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_3),
                           z=np.imag(x_f_2_downsamp_3),
                           mode='markers', name=f'Channel 2',
                           marker=dict(size=5, symbol="square")))  
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_3),
                           z=np.imag(x_f_3_downsamp_3),
                           mode='markers', name=f'Channel 3',
                           marker=dict(size=5, symbol="diamond")))  
fig3.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_3),
                           z=np.imag(x_f_4_downsamp_3),
                           mode='markers', name=f'Channel 4',
                           marker=dict(size=5, symbol="cross")))


fig3.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 3 (delay=2)"
)

# Zero axes

fig3.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig3.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig3.show()

In [104]:
### downsample into 4 staggered streams
# stream 1
idx = np.arange(1000)
x_t_total_downsamp_4 = x_t_total[3::4]
x_t_1_downsamp_4 = x_t_1[3::4]
x_t_2_downsamp_4 = x_t_2[3::4]
x_t_3_downsamp_4 = x_t_3[3::4]
x_t_4_downsamp_4 = x_t_4[3::4]

x_f_total_downsamp_4 = fft(x_t_total_downsamp_4)/len(x_t_total_downsamp_4)
x_f_1_downsamp_4 = fft(x_t_1_downsamp_4)/len(x_t_1_downsamp_4)
x_f_2_downsamp_4 = fft(x_t_2_downsamp_4)/len(x_t_2_downsamp_4)
x_f_3_downsamp_4 = fft(x_t_3_downsamp_4)/len(x_t_3_downsamp_4)
x_f_4_downsamp_4 = fft(x_t_4_downsamp_4)/len(x_t_4_downsamp_4)

x_f_total_downsamp_4[abs(x_f_total_downsamp_4)<1e-3] = np.nan
x_f_1_downsamp_4[abs(x_f_1_downsamp_4)<1e-3] = np.nan
x_f_2_downsamp_4[abs(x_f_2_downsamp_4)<1e-3] = np.nan
x_f_3_downsamp_4[abs(x_f_3_downsamp_4)<1e-3] = np.nan
x_f_4_downsamp_4[abs(x_f_4_downsamp_4)<1e-3] = np.nan


fig4 = go.Figure()
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_1_downsamp_4),
                           z=np.imag(x_f_1_downsamp_4),
                           mode='markers', name=f'Channel 1',
                           marker=dict(size=5, symbol="circle")))    
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_2_downsamp_4),
                           z=np.imag(x_f_2_downsamp_4),
                           mode='markers', name=f'Channel 2',
                           marker=dict(size=5, symbol="square")))  
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_3_downsamp_4),
                           z=np.imag(x_f_3_downsamp_4),
                           mode='markers', name=f'Channel 3',
                           marker=dict(size=5, symbol="diamond")))  
fig4.add_trace(go.Scatter3d(x=idx,
                           y=np.real(x_f_4_downsamp_4),
                           z=np.imag(x_f_4_downsamp_4),
                           mode='markers', name=f'Channel 4',
                           marker=dict(size=5, symbol="cross")))


fig4.update_layout(width=1000,height=800,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1.5, 1.5], title="Y"),
        zaxis=dict(range=[-1.5, 1.5], title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-1, y=-1, z=1)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 4 (delay=3)",
)

# Zero axes

fig4.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig4.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig4.show()

In [105]:
# --- Combine them into one figure with frames ---
combined = go.Figure()

# Add the first figure's traces as the initial data
combined.add_traces(fig1.data)

# Build frames from all figs
frames = []
for i, f in enumerate([fig1, fig2, fig3, fig4], start=1):
    frames.append(go.Frame(data=f.data, name=f"frame{i}"))

combined.frames = frames

# Add slider
combined.update_layout(
    sliders=[{
        "steps": [
            {"args": [[f.name], {"frame": {"duration": 0}, "mode": "immediate"}],
             "label": f"Stream {i+1} (delay={i})", "method": "animate"}
            for i, f in enumerate(combined.frames)
        ],
        "transition": {"duration": 0},
        "x": 0.1, "y": -0.1, "len": 0.9
    }],
    width=700,height=650,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(range=[-1, 1], title="I"),
        zaxis=dict(range=[-1, 1], title="Q"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.75)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Staggered Down-sampled Streams",
    legend=dict(
        orientation="h",          # horizontal
        yanchor="bottom",         # anchor legend to bottom
        y=0,                   # push legend below plot
        xanchor="center",         # anchor centered horizontally
        x=0.5
    )
)

combined.show()

In [29]:
combined.write_html("staggered_downsampled_streams.html",
               include_plotlyjs='cdn',
               full_html=False)

# Decompose the Filter

In [89]:
num_taps = 64  # Number of filter taps (coefficients) - typically odd for linear phase

filter_taps = signal.firwin(num_taps, 0.25, pass_zero=True)

# filter_taps = signal.resample(filter_taps, 4000)

# filter_freq_response = fft(filter_taps, 4000)/len(filter_taps)
# # filter_freq_response /= np.max(np.abs(filter_freq_response))
# filter_taps = ifft(filter_freq_response,64)*len(filter_freq_response)

In [91]:
filter_1_t = filter_taps[0::4]
filter_2_t = filter_taps[1::4]
filter_3_t = filter_taps[2::4]
filter_4_t = filter_taps[3::4]

filter_1_f = fft(filter_1_t, 1000)*4
filter_2_f = fft(filter_2_t, 1000)*4
filter_3_f = fft(filter_3_t, 1000)*4
filter_4_f = fft(filter_4_t, 1000)*4

In [125]:
out_ch1_1 = filter_1_f*x_f_1_downsamp_1 + filter_1_f*x_f_1_downsamp_1 + filter_1_f*x_f_1_downsamp_1 + filter_1_f*x_f_1_downsamp_1
out_ch2_1 = filter_1_f*x_f_2_downsamp_1 + filter_1_f*x_f_2_downsamp_1 + filter_1_f*x_f_2_downsamp_1 + filter_1_f*x_f_2_downsamp_1
out_ch3_1 = filter_1_f*x_f_3_downsamp_1 + filter_1_f*x_f_3_downsamp_1 + filter_1_f*x_f_3_downsamp_1 + filter_1_f*x_f_3_downsamp_1
out_ch4_1 = filter_1_f*x_f_4_downsamp_1 + filter_1_f*x_f_4_downsamp_1 + filter_1_f*x_f_4_downsamp_1 + filter_1_f*x_f_4_downsamp_1


In [126]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(out_ch1_1),
                           z=np.imag(out_ch1_1),
                           mode='markers', name=f'Channel 1',
                           marker=dict(size=5, symbol="circle")))    
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(out_ch2_1),
                           z=np.imag(out_ch2_1),
                           mode='markers', name=f'Channel 2',
                           marker=dict(size=5, symbol="square")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(out_ch3_1),
                           z=np.imag(out_ch3_1),
                           mode='markers', name=f'Channel 3',
                           marker=dict(size=5, symbol="diamond")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(out_ch4_1),
                           z=np.imag(out_ch4_1),
                           mode='markers', name=f'Channel 4',
                           marker=dict(size=5, symbol="cross")))


fig1.update_layout(width=1000,height=600,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(title="Y"),
        zaxis=dict(title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.75)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 1 (delay=0)"
)

# Zero axes

fig1.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig1.show()

In [122]:
fig1 = go.Figure()

sig1 = x_f_1_downsamp_1 + x_f_1_downsamp_2 + x_f_1_downsamp_3 + x_f_1_downsamp_4
sig2 = x_f_2_downsamp_1 + x_f_2_downsamp_2 + x_f_2_downsamp_3 + x_f_2_downsamp_4
sig3 = x_f_3_downsamp_1 + x_f_3_downsamp_2 + x_f_3_downsamp_3 + x_f_3_downsamp_4
sig4 = x_f_4_downsamp_1 + x_f_4_downsamp_2 + x_f_4_downsamp_3 + x_f_4_downsamp_4

fig1 = go.Figure()
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(sig1),
                           z=np.imag(sig1),
                           mode='markers', name=f'Channel 1',
                           marker=dict(size=5, symbol="circle")))    
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(sig2),
                           z=np.imag(sig2),
                           mode='markers', name=f'Channel 2',
                           marker=dict(size=5, symbol="square")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(sig3),
                           z=np.imag(sig3),
                           mode='markers', name=f'Channel 3',
                           marker=dict(size=5, symbol="diamond")))  
fig1.add_trace(go.Scatter3d(x=idx,
                           y=np.real(sig4),
                           z=np.imag(sig4),
                           mode='markers', name=f'Channel 4',
                           marker=dict(size=5, symbol="cross")))


fig1.update_layout(width=1000,height=600,
    scene=dict(
        xaxis=dict( title="Frequency Samples"),
        yaxis=dict(title="Y"),
        zaxis=dict(title="Z"),
        aspectmode="manual",           # allow manual control
        aspectratio=dict(x=1, y=0.5, z=0.5),  # scale of each axis
        camera=dict(eye=dict(x=-0.75, y=-1.5, z=0.75)),  # adjust the camera so labels are visible

    ),
    title="Decomposed Spectrum of Stream 1 (delay=0)"
)

# Zero axes

fig1.add_trace(go.Scatter3d(
    x=[0,1000], y=[0,0], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[0,0], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[-2,2], z=[0,0],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))

fig1.add_trace(go.Scatter3d(
    x=[1000,1000], y=[0,0], z=[-2,2],
    mode="lines", line=dict(color="black", width=4),
    name="X=0 axis", showlegend=False
))


fig1.show()

# Scratch 

In [217]:
x = np.random.randn(12)
h = np.random.randn(6)

y_filt_down = ifft(fft(x,12)*fft(h,12))
y_filt_down = y_filt_down[0::3]

h0 = h[0::3]
h1 = h[1::3]
h2 = h[2::3]

x0 = x[0::3]
x1 = x[1::3]
x2 = x[2::3]

y0 = ifft(fft(x0,4)*fft(h0,4))
y1 = ifft(fft(x2,4)*fft(h1,4))
y2 = ifft(fft(x1,4)*fft(h2,4))

y_filt_poly = np.roll(y0, 0) + np.roll(y1, 1)  + np.roll(y2, 1)